In [1]:
import pandas as pd
df = pd.read_csv('reviews_ratings.csv')
print(len(df))
df.head()

1119316


,Unnamed: 0,listing_id,date,reviewer_id,comments,clean_comment,sentiment,score,rating
0,0,2595,2018-04-29,128182033,We really enjoyed our stay at Jennifer’s apart...,stay jennifer apartment hospitable answered qu...,Positive,0.4588,4
1,1,2595,2018-06-06,115721479,Jennifer is very nice and responsive. The loca...,jennifer nice responsive location apt weird sl...,Positive,0.8074,5
2,2,5121,2012-10-30,3588281,"I had the most beautiful stay, \r\nGaron is a ...",beautiful stay special person absolute angel f...,Positive,0.9493,5
3,3,5203,2014-11-24,22508539,What can i say about my trip with these family...,trip family actuallyone word enough describe f...,Positive,0.9571,5
4,4,6848,2009-12-06,44462,We stayed only a (too) short weekend. It was r...,short weekend nice first experience airbnb eve...,Positive,0.9042,5


### take average of all rating of a listing

In [3]:
df_source = df.groupby('listing_id', as_index=False)['rating'].mean()
print(len(df_source))
df_source.head()

39848


,listing_id,rating
0,2060,5.000000
1,2595,4.511628
2,3831,4.289575
3,5099,4.236111
4,5114,4.693333


In [4]:
df_target=pd.read_csv('project/Clean_listings.csv',usecols=["id","review_scores_rating"])
print(len(df_target))
df_target.head()

50796


,id,review_scores_rating
0,2060,80.0
1,2595,94.0
2,3831,90.0
3,5099,90.0
4,5114,94.0


In [5]:
df_source.columns = ['id', 'predicted_rating']
df_target.columns = ['id', 'actual_rating']

### scaling predicted_rating to 1-100 

In [7]:
min=0
max=5
a=1
b=100
new_rating=[]
for i in range(len(df_source)):
    value = (((b-a)*(df_source.iloc[i].predicted_rating-min))/(max-min))+a
    new_rating.append(value)
df_source["predicted_rating"]=new_rating

In [9]:
df_source.head()

,id,predicted_rating
0,2060,100.000000
1,2595,90.330233
2,3831,85.933591
3,5099,84.875000
4,5114,93.928000


In [10]:
print(df_source['id'].duplicated().any())
print(df_target['id'].duplicated().any())

False
False


In [11]:
merged_df = pd.merge(df_source,df_target, on="id", how="inner")
merged_df.head()

,id,predicted_rating,actual_rating
0,2060,100.000000,80.0
1,2595,90.330233,94.0
2,3831,85.933591,90.0
3,5099,84.875000,90.0
4,5114,93.928000,94.0


In [14]:
len(merged_df)

39848

In [20]:
merged_df["diff_score"]=merged_df["actual_rating"]-merged_df["predicted_rating"]

In [21]:
merged_df.head()

,id,predicted_rating,actual_rating,diff_score
0,2060,100.000000,80.0,-20.000000
1,2595,90.330233,94.0,3.669767
2,3831,85.933591,90.0,4.066409
3,5099,84.875000,90.0,5.125000
4,5114,93.928000,94.0,0.072000


### Mean Absolute Error

In [36]:
import math
merged_df["square_diff"]= merged_df["diff_score"]*merged_df["diff_score"]
summ = merged_df["square_diff"].sum()
error=math.sqrt(summ/len(merged_df))
print(error)

15.750481398764286


In [16]:
sum_actual_rating = merged_df["actual_rating"].sum()
sum_actual_rating

3652029.0

In [17]:
sum_predicted_rating = merged_df["predicted_rating"].sum()
sum_predicted_rating

3336710.1009258498

### Mean Absolute Percent Error (MAPE) 

In [19]:
mape = 100*((sum_actual_rating - sum_predicted_rating)/sum_actual_rating)
print("Mean Absolute Percent Error=",mape)

Mean Absolute Percent Error= 8.634074348099379
